In [185]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import biosppy
import biosppy.signals.ecg as ecg
import numpy as np
import pyhrv.tools as tools
from pyhrv.hrv import hrv
import matplotlib
matplotlib.use('Qt4Agg')

# Activate interactive mode
import matplotlib.pyplot as plt
plt.ion()
import heartpy as hp
from xgboost import XGBClassifier,XGBRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import HistGradientBoostingClassifier

In [72]:
df_train_features=pd.read_csv('X_train.csv')
Y=pd.read_csv('y_train.csv')
x_test_f=pd.read_csv('X_test.csv')

In [73]:
df_train_features

,id,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x17797,x17798,x17799,x17800,x17801,x17802,x17803,x17804,x17805,x17806
0,0,-13,-9,-6,-4,0,2,6,12,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,-34,110,249,390,527,639,721,777,823,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,-34,-36,-37,-39,-41,-42,-44,-46,-48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,292,298,303,310,320,336,354,377,405,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,157,179,195,210,217,222,226,228,231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5112,5112,-247,-271,-285,-303,-334,-376,-413,-432,-443,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5113,5113,62,62,61,61,61,61,61,61,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5114,5114,-95,-110,-124,-131,-126,-114,-95,-67,-42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5115,5115,-50,-48,-45,-42,-38,-35,-32,-30,-28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# Creates Train & Validation set with the relevant Data

def Vector(feature,label):
    label=label.iloc[:,1]
    feature=feature.iloc[:,1:]
    #X_train, X_test, y_train, y_test  = train_test_split(feature,label, test_size=0.2, random_state=222)
    return feature,label

In [75]:
X_values,Y_labels= Vector(df_train_features,Y)

In [76]:
#np.size(X_train,axis=0)

In [77]:
features=np.zeros((np.size(X_train,axis=0),5))
testfeatures=np.zeros((np.size(x_test_f,axis=0),5))

In [212]:
def Extraction(X_train):
    results_all= np.zeros((np.size(X_train,axis=0),19))
    for i in range(np.size(X_train,axis=0)):
        feature=X_train.iloc[i,:]
        feature=feature.dropna(axis=0)
        signal1=np.array(feature)
        signal1=signal1.flatten()
        #peaks=ecg.ecg(signal=signal1, sampling_rate=300.0, show=False)
        #t, filtered_signal, rpeaks = biosppy.signals.ecg.ecg(feature, show=False)[:3]
        #nni = tools.nn_intervals(t[rpeaks])
        #analyze_df = hrv(rpeaks=t[rpeaks], sampling_rate=300, plot_ecg=False,plot_tachogram=False, show=False)
        wd, m = hp.process(signal1, sample_rate = 300.0, bpmmin = 40, bpmmax = 100)
        z=0
        for measure in m.keys():
                results_all[i,z] = m[measure]
                z=z+1
                if z==12:
                    results_all[i,13] = np.amax(signal1)
                    results_all[i,14] = np.amin(signal1)
                    results_all[i,15] = np.median(signal1)
                    results_all[i,16] = np.mean(signal1)
                    results_all[i,17] = np.std(signal1)
                    results_all[i,18] = np.var(signal1)
        if i==3500:
            print(3500)
            print(m.keys())
    return results_all
        

In [ ]:
def Extraction2(X_train):
    #results_all= np.zeros((np.size(X_train,axis=0),13))
    for i in range(np.size(X_train,axis=0)):
        feature=X_train.iloc[i,:]
        feature=feature.dropna(axis=0)
        signal1=np.array(feature)
        signal1=signal1.flatten()
        #peaks=ecg.ecg(signal=signal1, sampling_rate=300.0, show=False)
        #t, filtered_signal, rpeaks = biosppy.signals.ecg.ecg(feature, show=False)[:3]
        #nni = tools.nn_intervals(t[rpeaks])
        #analyze_df = hrv(rpeaks=t[rpeaks], sampling_rate=300, plot_ecg=False,plot_tachogram=False, show=False)
        wd, m = hp.process(signal1, sample_rate = 300.0, bpmmin = -1000, bpmmax = 1000)
        
        if i==0:
            m=pd.DataFrame(m.values() ,index=m.keys())
            result=m.T
        else:
            
            m=pd.DataFrame(m.values(),index=m.keys())
            
            result=result.append(m.T)
        if i==1000:
            print(1000)
            print(m.keys())
    return result
        

In [213]:
X_features=Extraction(X_values)

C:\Users\pwili\AppData\Local\Temp/ipykernel_17644/1641713810.py:15: UserWarning: Warning: converting a masked element to nan.
  results_all[i,z] = m[measure]


3500
dict_keys(['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate'])


In [214]:
X_features

array([[ 2.31101647e+02,  2.59626016e+02,  1.44519842e+02, ...,
         9.16431810e+00,  1.08995377e+02,  1.18799921e+04],
       [ 2.87795346e+02,  2.08481481e+02,  1.41586392e+02, ...,
         1.02739390e+01,  1.38783053e+02,  1.92607358e+04],
       [ 6.66226041e+01,  9.00595238e+02,  3.57799000e+01, ...,
         1.55421277e+01,  1.99155837e+02,  3.96630475e+04],
       ...,
       [ 2.13276231e+02,  2.81325301e+02,  1.70847905e+02, ...,
        -1.39450083e+00,  1.93885203e+02,  3.75914721e+04],
       [ 7.14194292e+01,  8.40107527e+02,  5.66539130e+01, ...,
         7.78669065e+00,  7.02175324e+01,  4.93050186e+03],
       [ 3.34929539e+02,  1.79142157e+02,  1.23505442e+02, ...,
        -1.30841328e+01,  1.27239498e+02,  1.61898898e+04]])

In [215]:
#X_train, X_test, y_train, y_test  = train_test_split(X_features,Y_labels, test_size=0.2, random_state=222)

In [216]:
X_train=X_features
y_train=Y_labels

In [217]:
model3=HistGradientBoostingClassifier(learning_rate=0.05)

In [218]:
X_train

array([[ 2.31101647e+02,  2.59626016e+02,  1.44519842e+02, ...,
         9.16431810e+00,  1.08995377e+02,  1.18799921e+04],
       [ 2.87795346e+02,  2.08481481e+02,  1.41586392e+02, ...,
         1.02739390e+01,  1.38783053e+02,  1.92607358e+04],
       [ 6.66226041e+01,  9.00595238e+02,  3.57799000e+01, ...,
         1.55421277e+01,  1.99155837e+02,  3.96630475e+04],
       ...,
       [ 2.13276231e+02,  2.81325301e+02,  1.70847905e+02, ...,
        -1.39450083e+00,  1.93885203e+02,  3.75914721e+04],
       [ 7.14194292e+01,  8.40107527e+02,  5.66539130e+01, ...,
         7.78669065e+00,  7.02175324e+01,  4.93050186e+03],
       [ 3.34929539e+02,  1.79142157e+02,  1.23505442e+02, ...,
        -1.30841328e+01,  1.27239498e+02,  1.61898898e+04]])

In [219]:
model3.fit(X_train,y_train)

HistGradientBoostingClassifier(learning_rate=0.05)

In [221]:
#X_values,Y_labels= Vector(df_train_features,Y)

In [222]:
X_test_features=Extraction(x_test_f.iloc[:,1:])

C:\Users\pwili\AppData\Local\Temp/ipykernel_17644/1641713810.py:15: UserWarning: Warning: converting a masked element to nan.
  results_all[i,z] = m[measure]


In [223]:
X_test_features

array([[6.97481317e+01, 8.60238095e+02, 8.35992356e+01, ...,
        3.38170328e+01, 2.95169594e+02, 8.71250892e+04],
       [5.58904110e+01, 1.07352941e+03, 6.92392894e+01, ...,
        2.57082114e+01, 1.54982133e+02, 2.40194616e+04],
       [2.99931522e+02, 2.00045662e+02, 1.51035485e+02, ...,
        3.86244674e+00, 2.57362051e+02, 6.62352254e+04],
       ...,
       [1.61644382e+02, 3.71185185e+02, 1.80464634e+02, ...,
        1.28195295e+01, 1.32633911e+02, 1.75917543e+04],
       [8.22708684e+01, 7.29298246e+02, 9.49477834e+00, ...,
        1.23710841e+01, 9.42972716e+01, 8.89197543e+03],
       [1.02084611e+02, 5.87747748e+02, 1.20654009e+02, ...,
        1.97690540e+01, 2.54342368e+02, 6.46900399e+04]])

In [224]:
Y_final=model3.predict(X_test_features)

In [225]:
# Create a data frame with the data for the submission
d = {'id': x_test_f.iloc[:,0], 'y': Y_final}

df = pd.DataFrame(data=d)

# Store the data in a csv file
df.to_csv('prediction3.csv', index=False)